In [1]:
#import packages
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
# suppress warning
import warnings
warnings.filterwarnings('ignore')
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import scipy
from sklearn.metrics import r2_score
from statsmodels.graphics.tsaplots import plot_acf

In [27]:
# import csvs
complaints_CD = pd.read_csv('complaints_by_CD.csv')
complaints_type_CD = pd.read_csv('complaints_by_type_CD.csv')
aq_CD = pd.read_csv('airquality_seasonal_cd.csv')

In [28]:
aq_CD.head()

UniqueID  IndicatorID                             Name Measure  \
0    169573          365  Fine Particulate Matter (PM2.5)    Mean   
1    547775          365  Fine Particulate Matter (PM2.5)    Mean   
2    606069          365  Fine Particulate Matter (PM2.5)    Mean   
3    168511          365  Fine Particulate Matter (PM2.5)    Mean   
4    168688          365  Fine Particulate Matter (PM2.5)    Mean   

           MeasureInfo GeoTypeName  GeoJoinID  \
0  mcg per cubic meter          CD        503   
1  mcg per cubic meter          CD        503   
2  mcg per cubic meter          CD        503   
3  mcg per cubic meter          CD        503   
4  mcg per cubic meter          CD        503   

                        GeoPlaceName  Year  Season  Start_Date  DataValue  
0  Tottenville and Great Kills (CD3)  2013  Summer  06/01/2013       9.13  
1  Tottenville and Great Kills (CD3)  2017  Winter  12/01/2016       6.87  
2  Tottenville and Great Kills (CD3)  2018  Summer  06/01/2018       7.36  
3  Tottenville and Great Kills (CD3)  2010  Winter  12/01/2009       9.07  
4  Tottenville and Great Kills (CD3)  2013  Winter  12/01/2012       9.60

In [29]:
complaints_CD = complaints_CD.drop(columns = ['Unnamed: 0'])
complaints_CD.columns = ['year', 'season', 'BoroCD', 'Total AQ Complaints']

In [30]:
complaints_CD.loc[complaints_CD['season']=='summer', 'season'] = 'Summer'
complaints_CD.loc[complaints_CD['season']=='winter', 'season'] = 'Winter'

In [31]:
complaints_CD.head()

year  season  BoroCD  Total AQ Complaints
0  2010  Summer   101.0                   70
1  2010  Summer   102.0                  134
2  2010  Summer   103.0                  124
3  2010  Summer   104.0                  146
4  2010  Summer   105.0                  151

In [33]:
aq_CD = aq_CD.merge(complaints_CD, left_on = ['Year', 'Season', 'GeoJoinID'], right_on = ['year','season','BoroCD'])

In [34]:
aq_CD.head()

UniqueID  IndicatorID                             Name Measure  \
0    169573          365  Fine Particulate Matter (PM2.5)    Mean   
1    166918          375           Nitrogen Dioxide (NO2)    Mean   
2    216630          386                       Ozone (O3)    Mean   
3    547775          365  Fine Particulate Matter (PM2.5)    Mean   
4    550167          375           Nitrogen Dioxide (NO2)    Mean   

           MeasureInfo GeoTypeName  GeoJoinID  \
0  mcg per cubic meter          CD        503   
1                  ppb          CD        503   
2                  ppb          CD        503   
3  mcg per cubic meter          CD        503   
4                  ppb          CD        503   

                        GeoPlaceName  Year  Season  Start_Date  DataValue  \
0  Tottenville and Great Kills (CD3)  2013  Summer  06/01/2013       9.13   
1  Tottenville and Great Kills (CD3)  2013  Summer  06/01/2013       7.59   
2  Tottenville and Great Kills (CD3)  2013  Summer  06/01/2013      32.64   
3  Tottenville and Great Kills (CD3)  2017  Winter  12/01/2016       6.87   
4  Tottenville and Great Kills (CD3)  2017  Winter  12/01/2016      17.47   

   year  season  BoroCD  Total AQ Complaints  
0  2013  Summer   503.0                   10  
1  2013  Summer   503.0                   10  
2  2013  Summer   503.0                   10  
3  2017  Winter   503.0                  138  
4  2017  Winter   503.0                  138

In [36]:
complaints_type_CD = complaints_type_CD.drop(columns = ['Unnamed: 0'])
complaints_type_CD.columns = ['year','season', 'BoroCD', 'Descriptor', 'Complaints']
complaints_type_CD.loc[complaints_type_CD['season']=='summer', 'season'] = 'Summer'
complaints_type_CD.loc[complaints_type_CD['season']=='winter', 'season'] = 'Winter'

In [37]:
complaints_type_CD.head()

year  season  BoroCD                                Descriptor  Complaints
0  2010  Summer   101.0               Air: Dust, Commercial (AE2)           3
1  2010  Summer   101.0  Air: Dust, Construction/Demolition (AE4)          11
2  2010  Summer   101.0              Air: Dust, Residential (AE1)           1
3  2010  Summer   101.0       Air: Odor/Fumes, Dry Cleaners (AD1)           1
4  2010  Summer   101.0    Air: Odor/Fumes, Private Carting (AD4)           1

In [42]:
#complaints_type_CD.pivot(index = ['year','season', 'BoroCD'], columns = 'Descriptor', values = 'Complaints')
complaints_type_CD = (complaints_type_CD.set_index(['year','season','BoroCD','Descriptor'])['Complaints']
       .unstack()
       .reset_index()
       .rename_axis(None, axis=1))

In [44]:
aq_CD = aq_CD.merge(complaints_type_CD, left_on = ['Year', 'Season', 'GeoJoinID'], right_on = ['year','season','BoroCD'])

In [45]:
aq_CD.columns

Index(['UniqueID', 'IndicatorID', 'Name', 'Measure', 'MeasureInfo',
       'GeoTypeName', 'GeoJoinID', 'GeoPlaceName', 'Year', 'Season',
       'Start_Date', 'DataValue', 'year_x', 'season_x', 'BoroCD_x',
       'Total AQ Complaints', 'year_y', 'season_y', 'BoroCD_y',
       'Air: Dust, Commercial (AE2)',
       'Air: Dust, Construction/Demolition (AE4)',
       'Air: Dust, Other (Use Comments) (AE5)', 'Air: Dust, Residential (AE1)',
       'Air: Odor, Nail Salon (AD8)',
       'Air: Odor, Sweet From Unknown Source (AZ1)',
       'Air: Odor/Fumes, Dry Cleaners (AD1)',
       'Air: Odor/Fumes, Other (Use Comments) (AD6)',
       'Air: Odor/Fumes, Private Carting (AD4)',
       'Air: Odor/Fumes, Restaurant (AD2)',
       'Air: Odor/Fumes, Vehicle Idling (AD3)',
       'Air: Open Fire, Commercial (AC2)',
       'Air: Open Fire, Construction/Demolition (AC4)',
       'Air: Open Fire, Residential (AC1)',
       'Air: Other Air Problem (Use Comments) (AZZ)',
       'Air: Smoke, Chimney or ve

In [47]:
aq_CD = aq_CD.drop(columns = ['year_x', 'season_x', 'BoroCD_x','year_y', 'season_y', 'BoroCD_y'])

In [48]:
aq_CD #dataframe with all complaint type counts, plus the aggregate counts, for each season and each CD

UniqueID  IndicatorID                             Name Measure  \
0       169573          365  Fine Particulate Matter (PM2.5)    Mean   
1       166918          375           Nitrogen Dioxide (NO2)    Mean   
2       216630          386                       Ozone (O3)    Mean   
3       547775          365  Fine Particulate Matter (PM2.5)    Mean   
4       550167          375           Nitrogen Dioxide (NO2)    Mean   
...        ...          ...                              ...     ...   
3063    547601          365  Fine Particulate Matter (PM2.5)    Mean   
3064    549993          375           Nitrogen Dioxide (NO2)    Mean   
3065    168571          365  Fine Particulate Matter (PM2.5)    Mean   
3066    165916          375           Nitrogen Dioxide (NO2)    Mean   
3067    165031          383             Sulfur Dioxide (SO2)    Mean   

              MeasureInfo GeoTypeName  GeoJoinID  \
0     mcg per cubic meter          CD        503   
1                     ppb          CD        503   
2                     ppb          CD        503   
3     mcg per cubic meter          CD        503   
4                     ppb          CD        503   
...                   ...         ...        ...   
3063  mcg per cubic meter          CD        101   
3064                  ppb          CD        101   
3065  mcg per cubic meter          CD        101   
3066                  ppb          CD        101   
3067                  ppb          CD        101   

                           GeoPlaceName  Year  Season  ...  \
0     Tottenville and Great Kills (CD3)  2013  Summer  ...   
1     Tottenville and Great Kills (CD3)  2013  Summer  ...   
2     Tottenville and Great Kills (CD3)  2013  Summer  ...   
3     Tottenville and Great Kills (CD3)  2017  Winter  ...   
4     Tottenville and Great Kills (CD3)  2017  Winter  ...   
...                                 ...   ...     ...  ...   
3063           Financial District (CD1)  2017  Winter  ...   
3064           Financial District (CD1)  2017  Winter  ...   
3065           Financial District (CD1)  2012  Winter  ...   
3066           Financial District (CD1)  2012  Winter  ...   
3067           Financial District (CD1)  2012  Winter  ...   

     Air: Open Fire, Commercial (AC2)  \
0                                 NaN   
1                                 NaN   
2                                 NaN   
3                                 NaN   
4                                 NaN   
...                               ...   
3063                              NaN   
3064                              NaN   
3065                              NaN   
3066                              NaN   
3067                              NaN   

      Air: Open Fire, Construction/Demolition (AC4)  \
0                                               NaN   
1                                               NaN   
2                                               NaN   
3                                               NaN   
4                                               NaN   
...                                             ...   
3063                                            NaN   
3064                                            NaN   
3065                                            NaN   
3066                                            NaN   
3067                                            NaN   

      Air: Open Fire, Residential (AC1)  \
0                                   NaN   
1                                   NaN   
2                                   NaN   
3                                   NaN   
4                                   NaN   
...                                 ...   
3063                                NaN   
3064                                NaN   
3065                                NaN   
3066                                NaN   
3067                                NaN   

      Air: Other Air Problem (Use Comments) (AZZ)  \
0                                             1

In [49]:
aq_CD.to_csv("AQ_with311_seasonal_CD.csv")